# Nível I – Carregamento, Tratamento e Modelagem Básica (Conjunto de dados de exercícios para membros de academia
)

## 1. Importar bibliotecas necessárias

In [51]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

## 2. Carregar o CSV e selecionar colunas

In [52]:
df = pd.read_csv('databases/gym_members_exercise_tracking.csv')
df = df[[
    'Age', 'Gender', 'Weight (kg)', 'Height (m)', 'Avg_BPM',
    'Resting_BPM', 'Session_Duration (hours)',
    'Workout_Frequency (days/week)', 'Water_Intake (liters)',
    'Experience_Level'
]]

## 3. Renomear colunas para português

 - Age → idade  
 - Gender → sexo  
 - Weight (kg) → peso_kg  
 - Height (m) → altura_m  
 - Avg_BPM → bpm_medio  
 - Resting_BPM → bpm_repouso  
 - Session_Duration (hours) → duracao_sessao_h  
 - Workout_Frequency (days/week) → frequencia_treino  
 - Water_Intake (liters) → ingestao_agua  
 - Experience_Level → nivel_experiencia 

In [53]:
df = df.rename(columns={
    'Age': 'idade',
    'Gender': 'sexo',
    'Weight (kg)': 'peso_kg',
    'Height (m)': 'altura_m',
    'Avg_BPM': 'bpm_medio',
    'Resting_BPM': 'bpm_repouso',
    'Session_Duration (hours)': 'duracao_sessao_h',
    'Workout_Frequency (days/week)': 'frequencia_treino',
    'Water_Intake (liters)': 'ingestao_agua',
    'Experience_Level': 'nivel_experiencia'
})
df.head()

,idade,sexo,peso_kg,altura_m,bpm_medio,bpm_repouso,duracao_sessao_h,frequencia_treino,ingestao_agua,nivel_experiencia
0,56,Male,88.3,1.71,157,60,1.69,4,3.5,3
1,46,Female,74.9,1.53,151,66,1.30,4,2.1,2
2,32,Female,68.1,1.66,122,54,1.11,4,2.3,2
3,25,Male,53.2,1.70,164,56,0.59,3,2.1,1
4,38,Male,46.1,1.79,158,68,0.64,3,2.8,1


 ## 4. Verificar e imputar valores faltantes

In [55]:
imputador = SimpleImputer(strategy='median')
col_num = [
    'idade','peso_kg','altura_m','bpm_medio','bpm_repouso',
    'duracao_sessao_h','frequencia_treino','ingestao_agua'
]
df[col_num] = imputador.fit_transform(df[col_num])

## 5. Codificar variáveis categóricas

 - `sexo`: Masculino/Feminino → 0/1  
 - `nivel_experiencia`: Beginner/Intermediate/Advanced → 0/1/2  

In [56]:
le_sexo = LabelEncoder()
df['sexo'] = le_sexo.fit_transform(df['sexo'])

le_nivel = LabelEncoder()
df['nivel_codigo'] = le_nivel.fit_transform(df['nivel_experiencia'])

## 6. Preparar X e y, e dividir em treino e teste

In [57]:
X = df[col_num + ['sexo']]
y = df['nivel_codigo']

X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## 7. Treinar e avaliar modelos dois modelos

In [59]:
modelos = {
    'Regressão Logística': LogisticRegression(solver='liblinear', max_iter=2000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

for nome, mdl in modelos.items():
    mdl.fit(X_treino, y_treino)
    preds = mdl.predict(X_teste)
    print(f"\n>>> {nome}")
    print(f"Acurácia: {accuracy_score(y_teste, preds):.4f}")
    print(classification_report(y_teste, preds))


>>> Regressão Logística
Acurácia: 0.8256
              precision    recall  f1-score   support

           0       0.74      0.92      0.82        75
           1       0.89      0.67      0.76        82
           2       0.93      0.97      0.95        38

    accuracy                           0.83       195
   macro avg       0.85      0.85      0.84       195
weighted avg       0.84      0.83      0.82       195


>>> Random Forest
Acurácia: 0.8769
              precision    recall  f1-score   support

           0       0.85      0.83      0.84        75
           1       0.85      0.87      0.86        82
           2       1.00      1.00      1.00        38

    accuracy                           0.88       195
   macro avg       0.90      0.90      0.90       195
weighted avg       0.88      0.88      0.88       195



# Nível II – Avaliação Repetida, Salvamento e Interface

In [60]:
import numpy as np
import pickle
import gradio as gr


## 8. Avaliar em 30 repetições

In [61]:
resultados = {'RegLog': [], 'RF': []}
for i in range(30):
    X_tr, X_ts, y_tr, y_ts = train_test_split(
        X, y, test_size=0.2, random_state=i, stratify=y)
    modelos['Regressão Logística'].fit(X_tr, y_tr)
    modelos['Random Forest'].fit(X_tr, y_tr)
    resultados['RegLog'].append(accuracy_score(y_ts, modelos['Regressão Logística'].predict(X_ts)))
    resultados['RF'].append(accuracy_score(y_ts, modelos['Random Forest'].predict(X_ts)))

## 9. Mostrar média e desvio-padrão

In [62]:
for nome, vals in resultados.items():
    print(f"{nome}: média={np.mean(vals):.4f}, std={np.std(vals, ddof=1):.4f}")

RegLog: média=0.8067, std=0.0255
RF: média=0.8827, std=0.0180


## 10. Salvar o melhor modelo

In [63]:
media_reg = np.mean(resultados['RegLog'])
media_rf  = np.mean(resultados['RF'])
best_name = 'Regressão Logística' if media_reg >= media_rf else 'Random Forest'
mdl_melhor = modelos[best_name]
mdl_melhor.fit(X, y)

with open('melhor_modelo_gym.pkl','wb') as f:
    pickle.dump((mdl_melhor, le_nivel), f)
print(f"Melhor modelo: {best_name}")

Melhor modelo: Random Forest


## 11. Interface Gradio

In [67]:
import numpy as np
import gradio as gr

def prever_experiencia(idade, peso_kg, altura_m, bpm_medio, bpm_repouso,
                       duracao_sessao_h, frequencia_treino, ingestao_agua, sexo_str):
    # Converte o sexo de string ("Male", "Female") para código (0 ou 1)
    sexo_codigo = le_sexo.transform([sexo_str])[0]
    
    # Monta o array de entrada na mesma ordem usada no treinamento
    entrada = np.array([[idade, peso_kg, altura_m, bpm_medio, bpm_repouso,
                         duracao_sessao_h, frequencia_treino, ingestao_agua, sexo_codigo]])
    
    # Faz a predição
    pred_codigo = mdl_melhor.predict(entrada)[0]
    
    # Converte o código de volta para o rótulo original (ex: "Beginner")
    mapeamento_nivel = {0: 'Beginner', 1: 'Intermediate', 2: 'Advanced'}
    return mapeamento_nivel.get(pred_codigo, f"Desconhecido: {pred_codigo}")


# Interface com Gradio
iface = gr.Interface(
    fn=prever_experiencia,
    inputs=[
        gr.Number(label="Idade"),
        gr.Number(label="Peso (kg)"),
        gr.Number(label="Altura (m)"),
        gr.Number(label="BPM Médio"),
        gr.Number(label="BPM Repouso"),
        gr.Number(label="Duração Sessão (h)"),
        gr.Number(label="Freq. Treino (dias/sem)"),
        gr.Number(label="Ingestão de Água (L)"),
        gr.Radio(choices=list(le_sexo.classes_), label="Sexo")  # Usa os nomes reais: 'Male', 'Female'
    ],
    outputs=gr.Textbox(label="Nível de Experiência"),
    title="Predição de Nível de Experiência na Academia",
    description="Informe os dados de um membro da academia para prever se ele é Beginner, Intermediate ou Advanced."
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


c:\Users\delom\Desktop\Programação\Desafio_IA_COMPET\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\delom\Desktop\Programação\Desafio_IA_COMPET\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\delom\Desktop\Programação\Desafio_IA_COMPET\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Using existing dataset file at: .gradio\flagged\dataset1.csv


## Tabela com exemplos de dados para teste

| Caso              | Idade | Peso (kg) | Altura (m) | BPM Médio | BPM Repouso | Duração Sessão (h) | Freq. Treino (dias/sem) | Ingestão de Água (L) | Sexo      | Saída Esperada |
| ----------------- | ----- | --------- | ---------- | --------- | ----------- | ------------------ | ----------------------- | -------------------- | --------- | -------------- |
| **Iniciante**     | 22    | 68        | 1.75       | 110       | 70          | 0.75               | 2                       | 1.2                  | Feminino  | Beginner       |
| **Intermediário** | 30    | 80        | 1.80       | 120       | 65          | 1.0                | 4                       | 2.0                  | Masculino | Intermediate   |
| **Avançado**      | 28    | 75        | 1.78       | 130       | 60          | 1.5                | 6                       | 2.5                  | Masculino | Advanced       |
| **Veterano**      | 40    | 85        | 1.70       | 125       | 62          | 1.25               | 5                       | 2.0                  | Feminino  | Advanced       |
